# Alexis Mesias 22562
# Julio Lemus   22461


In [2]:
import numpy as np
import altair as alt 
import pandas as pd

In [3]:
def init_parameters(n_features, n_neurons, n_output): 

    np.random.seed(100)
    W1 = np.random.uniform(size = (n_features, n_neurons))
    b1 = np.random.uniform(size = (1, n_neurons))

    W2 = np.random.uniform(size = (n_neurons, n_output))
    b2 = np.random.uniform(size = (1, n_output))

    return {
        "W1" : W1 
        , "b1" : b1 
        , "W2" : W2
        , "b2" : b2 
    }

In [4]:
def linear_function(W, X, b): 
    return (X @ W)+ b 

In [7]:
def cost_function(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))


In [8]:
def cost_function(A, y): 
    return (np.mean(np.power(A - y, 2)))/2 

In [10]:
def predict(X, W1, W2, b1, b2): 
    Z1 = linear_function(W1, X, b1)
    S1 = sigmoid_func(Z1)
    Z2 = linear_function(W2, S1, b2)
    S2 = sigmoid_func(Z2)
    return np.where(S2 >= 0.5, 1, 0)

In [11]:
def fit(X, y, n_features=2, n_neurons=3, n_output=1, iterations=10, eta=0.001): 
    params = init_parameters(
        n_features=n_features,
        n_neurons=n_neurons,
        n_output=n_output
    )

    errors = []

    for _ in range(iterations): 
        Z1 = linear_function(params['W1'], X, params['b1'])
        S1 = sigmoid_func(Z1)
        Z2 = linear_function(params['W2'], S1, params['b2'])
        S2 = sigmoid_func(Z2)

        error = cost_function(S2, y)
        errors.append(error)

        delta2 = S2 - y
        W2_gradients = S1.T @ delta2
        params["W2"] -= W2_gradients * eta
        params["b2"] -= np.sum(delta2, axis=0, keepdims=True) * eta

        delta1 = (delta2 @ params["W2"].T) * S1 * (1 - S1)
        W1_gradients = X.T @ delta1
        params["W1"] -= W1_gradients * eta
        params["b1"] -= np.sum(delta1, axis=0, keepdims=True) * eta

    return errors, params


In [12]:
y = np.array([[0, 1, 1, 0]]).T 
X = np.array([[0, 0, 1, 1]
              ,[0, 1, 0, 1]]).T 

In [13]:
errors, params = fit(X, y, iterations=5000, eta = 0.1)

In [14]:
y_pred = predict(X, params["W1"], params["W2"], params["b1"], params["b2"])
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100
print('Multi-layer perceptron accuracy: %.2f%%' % accuracy)


Multi-layer perceptron accuracy: 100.00%


In [ ]:
alt.data_transformers.disable_max_rows()
df = pd.DataFrame({"errors":errors, "time-step": np.arange(0, len(errors))})
alt.Chart(df).mark_line().encode(x="time-step", y="errors").properties(title='Chart 2')


alt.Chart(...)

# Mismo Dataset en Framework Pytorch

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import altair as alt

In [18]:
# Inputs: XOR problem
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T  # Shape (4, 2)
y = np.array([[0, 1, 1, 0]]).T  # Shape (4, 1)

# Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Define the model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = SimpleNet()

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
errors = []
epochs = 3000

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    errors.append(loss.item())

# Convert errors to DataFrame and plot
df2 = pd.DataFrame({"errors": errors, "time-step": np.arange(epochs)})

alt.Chart(df2).mark_line().encode(
    x="time-step",
    y="errors"
).properties(title='Chart 3')


alt.Chart(...)

# Preguntas


## 1. ¿Existen cambios de arquitectura en las 2 redes implementadas?
### No. Ambas son redes MLP con una capa oculta y una de salida. Solo cambia el número de neuronas y el framework utilizado (NumPy vs PyTorch).
## 2. ¿Exiten diferencias en la velocidad de convergencia entre las 2 redes?
### Sí. PyTorch converge más rápido gracias al optimizador Adam y operaciones optimizadas. NumPy usa gradiente simple y es más lento.
## 3. En la función init_parameters, ¿qué sucede si inicializamos los pesos en 0 en lugar de valores aleatorios? ¿y los bias?
### Pesos en 0: impide el aprendizaje (los nodos aprenden lo mismo).
### Bias en 0: no afecta negativamente, es común hacerlo.